# Per-User Retrieval | ユーザーごとの検索

検索アプリを構築する際には、多くの場合、複数のユーザーを考慮して構築する必要があります。これは、一人のユーザーだけでなく、多くの異なるユーザーのデータを保存している可能性があり、ユーザー同士が互いのデータを見えないようにする必要があることを意味します。つまり、特定の情報のみを取得するように検索チェーンを設定できるようにする必要があります。これには通常、二つのステップが関わっています。

> When building a retrieval app, you often have to build it with multiple users in mind. This means that you may be storing data not just for one user, but for many different users, and they should not be able to see eachother's data. This means that you need to be able to configure your retrieval chain to only retrieve certain information. This generally involves two steps.

**ステップ 1: 使用しているリトリーバーが複数ユーザーをサポートしていることを確認してください**

> **Step 1: Make sure the retriever you are using supports multiple users**

現時点でLangChainには、この機能に対する統一されたフラグやフィルターは存在しません。むしろ、各ベクトルストアやリトリーバーはそれぞれ独自のものを持っており、それらは異なる名称（ネームスペース、マルチテナンシーなど）で呼ばれることがあります。ベクトルストアでは、これは通常、`similarity_search`実行時に渡されるキーワード引数として利用可能です。使用しているリトリーバーが複数ユーザーをサポートしているかどうか、そしてサポートしている場合、その使用方法を知るためには、ドキュメントやソースコードを確認してください。

> At the moment, there is no unified flag or filter for this in LangChain. Rather, each vectorstore and retriever may have their own, and may be called different things (namespaces, multi-tenancy, etc). For vectorstores, this is generally exposed as a keyword argument that is passed in during `similarity_search`. By reading the documentation or source code, figure out whether the retriever you are using supports multiple users, and, if so, how to use it.

注意：サポートされていない、またはドキュメント化されていないリトリバーに対して、複数ユーザー向けのドキュメントやサポートを追加することは、LangChainへの素晴らしい貢献方法です。

> Note: adding documentation and/or support for multiple users for retrievers that do not support it (or document it) is a GREAT way to contribute to LangChain

**ステップ 2: そのパラメータをチェーンの設定可能なフィールドに追加する**

> **Step 2: Add that parameter as a configurable field for the chain**

これにより、実行時にチェーンを簡単に呼び出し、関連するフラグを設定することができます。設定に関する詳細は、[このドキュメント](docs/expression_language/how_to/configure)をご覧ください。

> This will let you easily call the chain and configure any relevant flags at runtime. See [this documentation](docs/expression_language/how_to/configure) for more information on configuration.

**ステップ 3: 設定可能なフィールドを使ってチェーンを呼び出す**

> **Step 3: Call the chain with that configurable field**

実行時に、設定可能なフィールドを持つこのチェーンを呼び出すことができます。

> Now, at runtime you can call this chain with configurable field.

## Code Example | コード例

コードでこれがどのように見えるかの具体的な例を見てみましょう。この例にはPineconeを使用します。

> Let's see a concrete example of what this looks like in code. We will use Pinecone for this example.



In [1]:
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

/Users/harrisonchase/.pyenv/versions/3.10.1/envs/langchain/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
# The environment should be the one specified next to the API key
# in your Pinecone console
pinecone.init(api_key="...", environment="...")
index = pinecone.Index("test-example")
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone(index, embeddings, "text")

vectorstore.add_texts(["i worked at kensho"], namespace="harrison")
vectorstore.add_texts(["i worked at facebook"], namespace="ankush")

['ce15571e-4e2f-44c9-98df-7e83f6f63095']

`namespace`の`pinecone`キーワード引数は、ドキュメントを分離するために使用できます

> The pinecone kwarg for `namespace` can be used to separate documents



In [6]:
# This will only get documents for Ankush
vectorstore.as_retriever(search_kwargs={"namespace": "ankush"}).get_relevant_documents(
    "where did i work?"
)

[Document(page_content='i worked at facebook')]

In [7]:
# This will only get documents for Harrison
vectorstore.as_retriever(
    search_kwargs={"namespace": "harrison"}
).get_relevant_documents("where did i work?")

[Document(page_content='i worked at kensho')]

これで、質問応答に使用するチェーンを作成することができます。

> We can now create the chain that we will use to do question-answering over



In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)

これは基本的な質問応答チェーンのセットアップです。

> This is basic question-answering chain set up.



In [10]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retriever = vectorstore.as_retriever()

ここでは、リトリーバーに設定可能なフィールドがあることを示しています。すべてのベクトルストアリトリーバーには`search_kwargs`というフィールドがあります。これはベクトルストア特有のフィールドを含む単なる辞書です。

> Here we mark the retriever as having a configurable field. All vectorstore retrievers have `search_kwargs` as a field. This is just a dictionary, with vectorstore specific fields



In [11]:
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

これで、設定可能なリトリバーを使用してチェーンを作成することができます

> We can now create the chain using our configurable retriever



In [12]:
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

これで、設定可能なオプションを持つチェーンを呼び出すことができます。`search_kwargs`は設定可能なフィールドのIDで、その値はPineconeで使用する検索のkwargsです。

> We can now invoke the chain with configurable options. `search_kwargs` is the id of the configurable field. The value is the search kwargs to use for Pinecone



In [13]:
chain.invoke(
    "where did the user work?",
    config={"configurable": {"search_kwargs": {"namespace": "harrison"}}},
)

'The user worked at Kensho.'

In [14]:
chain.invoke(
    "where did the user work?",
    config={"configurable": {"search_kwargs": {"namespace": "ankush"}}},
)

'The user worked at Facebook.'